# Sentiment analysis based on BOW and TF-IDF

In [77]:
import pandas as pd

import re

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## About the data

The data is taken from https://www.kaggle.com/datasets/yasserh/imdb-movie-ratings-sentiment-analysis.
It has two columns:
1. "text" - film reviews from IMDB, string;
2. "label" - int: 0 if the review was negative, 1 if the review was positive.

In [78]:
reviews = pd.read_csv('movie.csv')

In [79]:
reviews.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


## Data preprocessing

In [80]:
def prepare_stopwords(language='english'):
    """
    Prepares stopwords for a given language.
    """
    stop_words = stopwords.words(language)
    if language == 'english':
        stop_words.remove('not')
        stop_words.remove('no')
    return stop_words

In [81]:
def preprocess_string(text):
    """
    Preprocesses the text string in English for further BOW or TF-IDF analysis.
    After the preprocessing the string contains words only and is lower-cased, tokenized, 
    without stop words and lemmatized.
    param text: string to preprocess.
    return: a preprocessed string.
    """
    
    text = re.sub('[^a-zA-Z0-9]+', ' ', text)
    
    text = text.lower()
    
    tokens = word_tokenize(text)
    
    stop_words = prepare_stopwords()
    text = [word for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    
    text = ' '.join(text)
    
    return text

In [82]:
reviews['preprocessed_text'] = reviews['text'].apply(lambda text: preprocess_string(text))

In [83]:
reviews.head()

,text,label,preprocessed_text
0,I grew up (b. 1965) watching and loving the Th...,0,grew b 1965 watching loving thunderbird mate s...
1,"When I put this movie in my DVD player, and sa...",0,put movie dvd player sat coke chip expectation...
2,Why do people who do not know what a particula...,0,people not know particular time past like feel...
3,Even though I have great interest in Biblical ...,0,even though great interest biblical movie bore...
4,Im a die hard Dads Army fan and nothing will e...,1,im die hard dad army fan nothing ever change g...


## Splitting the data to train and test parts

In [84]:
X = reviews.drop(['label'], axis=1)
y = reviews['label']

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42, stratify=y)

## Text vectorizing

### BOW vectorizing

In [86]:
bow_vect = CountVectorizer()

In [87]:
X_train_bow = bow_vect.fit_transform(X_train['preprocessed_text'])
X_test_bow = bow_vect.transform(X_test['preprocessed_text'])

### TF-IDF vectorizing

In [88]:
tfidf_vect = TfidfVectorizer()

In [89]:
X_train_tfidf = tfidf_vect.fit_transform(X_train['preprocessed_text'])
X_test_tfidf = tfidf_vect.transform(X_test['preprocessed_text'])

## Naive Bayes classifier

In [90]:
nb_clf = MultinomialNB()

### BOW Naive Bayes

In [91]:
nb_clf.fit(X_train_bow, y_train)

MultinomialNB()

In [92]:
y_pred_bow_nb = nb_clf.predict(X_test_bow)

In [93]:
accuracy_score(y_test, y_pred_bow_nb) # 0.8556818181818182

0.8556818181818182

### TF-IDF Naive Bayes

In [94]:
nb_clf.fit(X_train_tfidf, y_train)

MultinomialNB()

In [95]:
y_pred_tfidf_nb = nb_clf.predict(X_test_tfidf)

In [96]:
accuracy_score(y_test, y_pred_tfidf_nb) # 0.8642424242424243

0.8642424242424243

## TF-IDF logistic regression

In [97]:
log_clf = LogisticRegression()

In [98]:
log_clf.fit(X_train_tfidf, y_train)

LogisticRegression()

In [99]:
y_pred_tfidf_log = log_clf.predict(X_test_tfidf)

In [100]:
accuracy_score(y_test, y_pred_tfidf_log) # 0.89 - best result

0.89

## Random forest

In [101]:
rf_clf = RandomForestClassifier()

### BOW random forest

In [102]:
rf_clf.fit(X_train_bow, y_train)

RandomForestClassifier()

In [103]:
y_pred_bow_rf = rf_clf.predict(X_test_bow)

In [104]:
accuracy_score(y_test, y_pred_bow_rf) # 0.8551515151515151

0.8551515151515151

### TF-IDF random forest

In [105]:
rf_clf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [106]:
y_pred_tfidf_rf = rf_clf.predict(X_test_tfidf)

In [107]:
accuracy_score(y_test, y_pred_tfidf_rf) # 0.8493181818181819

0.8493181818181819